In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from env import sql_connexion
import env

import acquire

# turn off pink warning boxes
import warnings
warnings.filterwarnings("ignore")

In [2]:
# obtaining the Zillow 2017 dataset for single-family homes

zil = acquire.get_zillow_data(env.sql_connexion)

In [ ]:
# now obtaining the data from the created .csv because SQL is taking too long with the internet.

# zil = pd.read_csv('zillow_single_family_properties_2017.csv')

In [3]:
zil.head()

,Unnamed: 0,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips,propertylandusetypeid
0,0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0,261.0
1,1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0,261.0
2,2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0,261.0
3,3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0,261.0
4,4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0,261.0


In [ ]:
# zil.to_csv('zillow_single_family_properties_2017.csv')

# converting to csv -- marking out this cell to avoid re-saving the file each time that I run the notebook

In [ ]:
zil.shape

# seeing the size of the dataframe

In [ ]:
zil.info()

# getting info on the unmanipulated dataframe

In [ ]:
zil.describe().T
# numerical data

In [ ]:
zil.columns.tolist()

#looking at columns
# will have to drop 'Unnamed: 0'

In [ ]:
zil.isna().sum()

#checking nulls in each column

In [ ]:
zil.columns[zil.isnull().any()].tolist()

# find column names with nulls

In [ ]:
# look at frequencies of each value in bedroomcnt

zil.bedroomcnt.value_counts()

# 3- and 4-br houses the most popular by far.

In [ ]:
zil.yearbuilt.value_counts()

# years built range from 1801 to 2016

In [4]:
zil.fips[zil.yearbuilt.isna()].value_counts()

# finding the fips codes and how many nulls in each

6037.0    3775
6111.0    2943
6059.0    2619
Name: fips, dtype: int64

In [ ]:
# dropping nulls because they're a small percentage of the overall data 

zil = zil.dropna()
zil.head()

**Null values consisted of less than 1.06pc of all entries, so the nulls were dropped.**

In [ ]:
(zil.bedroomcnt == 0).value_counts()

# 4397 houses have a bedroom count of 0 -- perhaps they're studios ?


In [ ]:
# # but what about houses with no bathrooms ?

(zil.bathroomcnt == 0).value_counts()

# # 4274 with no bathroom

In [ ]:
(zil.calculatedfinishedsquarefeet < 10).value_counts()


# all houses have a sq-footage, but 16 have a square footage of less than 16

In [ ]:
# renaming unsightly column names

zil = zil.rename(columns = {'bedroomcnt' : 'num_br', 'bathroomcnt' : 'num_ba', 
                            'calculatedfinishedsquarefeet' : 'total_sqft', 
                            'taxamount' : 'taxes', 'taxvaluedollarcnt' : 'current_tax_val', 
                            'yearbuilt' : 'year_built'})

In [ ]:
zil.head()

**The columns 'propertylandusetypeid' (the column was used solely to retrieve the data from SQL, and is now superfluous) and 'Unnamed: 0' are unnecessary and will be dropped.**

In [ ]:
# dropping unnecessary columns 'propertylandusetypeid' and 'Unnamed: 0'

zil = zil.drop(columns = ['propertylandusetypeid', 'Unnamed: 0'], axis = 0)
zil.head()

In [ ]:
zil.shape

# looking at dataset shape after wrangling

In [ ]:
## visualising the data

for col in zil.columns:
    sns.displot(zil[col], palette = ['red', 'green'], linewidth=2.5)
    plt.title(f'{col} distribution')
    plt.show()
    


**The graphs show that there is a normal distribution for num_br, num_ba and year_built (peaking in the 1960s). Total_sqft, current_tax_val and taxes are all very skewed distribution.**



In [ ]:
# Looking at boxplots of the data

for col in zil.columns:
    plt.figure(figsize = (8, 4))

    sns.boxplot(zil[col])
    plt.show()


**The data as shown in the boxplots appears just as concentrated around certain points as in the bar graphs. There are some definite outliers.**

.